In [22]:
import tensorflow as tf

import numpy as np
import os
import time

In [23]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [24]:
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')
print('텍스트의 길이: {}'.format(len(text)))

텍스트의 길이: 1115394


In [25]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [26]:
vocab = sorted(set(text))
print('고유 문자 수 : {}'.format(len(vocab)))

고유 문자 수 : 65


In [27]:
char2idx = {u : i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [28]:
print('{')
for char, _ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print(' ...\n')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
 ...



In [29]:
seq_len = 100
examples_per_epochs = len(text)//seq_len

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [30]:
sequences = char_dataset.batch(seq_len + 1, drop_remainder = True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [31]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [32]:
for input_example, target_example in dataset.take(1):
    print('input data : {}'.format(repr(''.join(idx2char[input_example.numpy()]))))
    print('target data : {}'.format(repr(''.join(idx2char[target_example.numpy()]))))

input data : 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
target data : 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [33]:
batch_size = 64
buffer_size = 10000

dataset = dataset.shuffle(buffer_size= buffer_size).batch(batch_size, drop_remainder= True)

print(dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>


In [34]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [35]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences= True, stateful= True, recurrent_initializer= 'glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])

    return model

In [36]:
model = build_model(vocab_size = vocab_size, embedding_dim= embedding_dim, rnn_units= rnn_units, batch_size = batch_size)

In [37]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [41]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy()

In [42]:
sampled_indices

array([13, 52, 54, 13, 32, 50, 39,  7, 27, 62, 25, 41, 38, 55, 44, 60, 27,
       63, 21, 51, 24, 14, 25, 28, 57, 22, 42, 39, 28,  8, 26, 16, 36, 54,
       51,  5,  1,  5, 28, 18, 57, 25,  4, 64, 48, 22, 49, 29, 36, 32, 19,
       59, 37, 55,  1, 31, 25, 36,  0, 40, 40, 28, 42, 50,  2, 23, 51, 56,
       20, 59,  9, 44, 56, 43,  8, 24, 21, 51, 27, 29, 20, 22,  3, 55, 14,
       40,  2, 24, 64, 61,  6, 33, 24, 20, 17,  2, 40, 57, 41, 12],
      dtype=int64)

In [43]:
print('input : \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('expected : \n', repr(''.join(idx2char[sampled_indices])))

input : 
 " seeth not this palpable device?\nYet who's so blind, but says he sees it not?\nBad is the world; and "

expected : 
 "AnpATla-OxMcZqfvOyImLBMPsJdaP.NDXpm' 'PFsM&zjJkQXTGuYq SMX\nbbPdl!KmrHu3fre.LImOQHJ$qBb!Lzw,ULHE!bsc?"


In [44]:
def loss(label, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(label, logits, from_logits= True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)

In [45]:
model.compile(optimizer= 'adam', loss = loss)

In [46]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_prefix,
    save_weights_only= True
)

In [47]:
epochs = 10

In [48]:
history = model.fit(dataset, epochs= epochs, callbacks= [checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 611s 4s/step - loss: 2.6117
Epoch 2/10
172/172 [==============================] - 569s 3s/step - loss: 1.9111
Epoch 3/10
172/172 [==============================] - 573s 3s/step - loss: 1.6588
Epoch 4/10
172/172 [==============================] - 585s 3s/step - loss: 1.5182
Epoch 5/10
172/172 [==============================] - 595s 3s/step - loss: 1.4344
Epoch 6/10
172/172 [==============================] - 591s 3s/step - loss: 1.3765
Epoch 7/10
172/172 [==============================] - 570s 3s/step - loss: 1.3292
Epoch 8/10
172/172 [==============================] - 550s 3s/step - loss: 1.2898
Epoch 9/10
172/172 [==============================] - 545s 3s/step - loss: 1.2539
Epoch 10/10
172/172 [==============================] - 541s 3s/step - loss: 1.2182


In [49]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [51]:
model = build_model(vocab_size= vocab_size, embedding_dim = embedding_dim, rnn_units = rnn_units, batch_size = 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [52]:
def generate_text(model, start_string):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
    
    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string= u"ROMEO: "))